 ### Read Data from kaggle

In [ ]:
!pip install kaggle
from google.colab import files
files.upload()
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c understanding_cloud_organization
!unzip understanding_cloud_organization.zip -d cloud_data

Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 5, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/build_env.py", line 19, in <module>
    from pip._internal.cli.spinners import open_spinner
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/spinners.py", line 9, in <module>
    from pip._internal.utils.logging import get_indentation
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/utils/logging.py", line 13, in <module>
    from pip._vendor.

TypeError: ignored

In [ ]:
!pip install keras

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# import plotting
from matplotlib import pyplot as plt
import matplotlib.patches as patches
import matplotlib
import seaborn as sns

# model
import cv2
import os
import albumentations as albu
import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input
from keras.layers import Conv2D, Conv2DTranspose
from keras.layers import MaxPooling2D
from keras.layers import concatenate
from keras.losses import binary_crossentropy
from keras.optimizers import Adam, Nadam
from keras.callbacks import Callback, ModelCheckpoint
from tqdm import tqdm
from sklearn.model_selection import train_test_split

### Preprocessing

In [ ]:
# load dataframe with train labels
train_df = pd.read_csv('cloud_data/train.csv')
train_df.sample(5)

,Image_Label,EncodedPixels
10380,75ccf10.jpg_Fish,50890 910 52290 910 53690 910 55090 910 56490 ...
21500,f78bab7.jpg_Fish,NaN
20612,ed48a59.jpg_Fish,1981228 1119 1982595 6 1982609 1 1982611 1 198...
3155,237043b.jpg_Sugar,21335 554 22735 554 24135 554 25535 554 26935 ...
11856,8734e73.jpg_Fish,NaN


In [ ]:
train_df.shape[0]

22184

In [ ]:
train_df.isnull().sum()

Image_Label          0
EncodedPixels    10348
dtype: int64

In [ ]:
# Split Image_Label into ImageId , Label and masked

train_df['ImageId'] = train_df['Image_Label'].apply(lambda x : x.split('_')[0])
train_df['Label'] = train_df['Image_Label'].apply(lambda x : x.split('_')[1])
train_df['hasMask'] = ~ train_df['EncodedPixels'].isna()

train_df.sample(5)

,Image_Label,EncodedPixels,ImageId,Label,hasMask
17260,c72878d.jpg_Fish,1090895 858 1092295 858 1093695 858 1095095 85...,c72878d.jpg,Fish,True
9889,701748e.jpg_Flower,589657 746 591025 3 591051 752 592425 5 592449...,701748e.jpg,Flower,True
3289,24f8df3.jpg_Flower,NaN,24f8df3.jpg,Flower,False
618,0799206.jpg_Gravel,NaN,0799206.jpg,Gravel,False
12489,8dfcd9f.jpg_Flower,NaN,8dfcd9f.jpg,Flower,False


In [ ]:
train_df['ImageId'].size

22184

In [ ]:
train_df.dropna(axis = 0,inplace=True)

In [ ]:
train_df.shape[0]

11836

### value_counts for image label

In [ ]:
train_df.loc[train_df['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().value_counts()

2    2372
3    1560
1    1348
4     266
Name: Image_Label, dtype: int64

### - Total training images is 5546, every images have four masks.(Fish, Flower, Gravel, Sugar), Total training dataset is 5546 * 4 = 22184

In [ ]:
import cv2
import base64

def decode_column(encoded_column):
    decoded_column = base64.b64decode(encoded_column.values.tobytes()).decode("utf-8", errors='ignore')
    return decoded_column

def resize_decode_column(encoded_column, new_width, new_height):
    # Decode the encoded column.
    decoded_column = decode_column(encoded_column)

    # Convert the decoded column to an image.
    image = cv2.imread(decoded_column[0])
    print(image)
    # Resize the image.
    resized_image = cv2.resize(image, (new_width, new_height))

    # Convert the resized image to base64 format.
    resized_column = base64.b64encode(resized_image).decode("utf-8")
    return resized_column

# Get the encoded column.
encoded_column = train_df["EncodedPixels"]

# Resize the decoded column to 320X480 pixels.
resized_decode_column = resize_decode_column(encoded_column, 320, 480)

# Save the resized decoded column to a new column.
train_df["resized_decode_column"] = resized_decode_column

None


error: ignored

## one hot encoding

In [ ]:
train_df = train_df[~train_df['EncodedPixels'].isnull()]
train_df['Image'] = train_df['Image_Label'].map(lambda x: x.split('_')[0])
train_df['Class'] = train_df['Image_Label'].map(lambda x: x.split('_')[1])

classes = train_df['Class'].unique()
train1_df = train_df.groupby('Image')['Class'].agg(set).reset_index()

for class_name in classes:
    train1_df[class_name] = train1_df['Class'].map(lambda x: 1 if class_name in x else 0)

train1_df.head()

In [ ]:
train1_df.shape[0]

In [ ]:
train1_df['Class'].value_counts()

## Note: 266 images has 4 type of cloud

In [ ]:
images=np.array(images)
train1_df['img']=list(images)
images.shape

In [ ]:
len(train1_df)

##**Split Data**

In [ ]:
# put types into y_train variable
Y= train1_df[['Fish','Flower','Sugar',	'Gravel']].values
# put 'image' column in x_train
X= train1_df['img'].values
X=np.array(X)
X=np.stack(X)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
print("shape x train",X_train.shape,"shape x test",X_test.shape,"shape y train",y_train.shape,"shape y test",y_test.shape)

### Data Generator

In [ ]:
# data augmentation
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=20,  # randomly rotate images in the range 5 degrees
        zoom_range = 0.1, # Randomly zoom image 10%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True)  # randomly flip images

datagen.fit(X_train)

In [ ]:
def get_model(img_size, num_classes):
    inputs = keras.Input(shape=img_size + (3,))

    ### [First half of the network: downsampling inputs] ###

    # Entry block
    x = keras.layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [64, 128, 256]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.SeparableConv2D(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = keras.layers.Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ### [Second half of the network: upsampling inputs] ###

    for filters in [256, 128, 64, 32]:
        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.Activation("relu")(x)
        x = keras.layers.Conv2DTranspose(filters, 3, padding="same")(x)
        x = keras.layers.BatchNormalization()(x)

        x = keras.layers.UpSampling2D(2)(x)

        # Project residual
        residual = keras.layers.UpSampling2D(2)(previous_block_activation)
        residual = keras.layers.Conv2D(filters, 1, padding="same")(residual)
        x = keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    outputs = keras.layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

    # Define the model
    model = keras.Model(inputs, outputs)

    return model

In [ ]:
model=get_model((480, 320),4)

In [ ]:
model.summary()

In [ ]:
train_df.sample()


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(1e-4),loss="categorical_crossentropy",metrics=["accuracy"])
from keras.callbacks import  EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5,restore_best_weights=True)
# Train the model, doing validation at the end of each epoch.
history = model.fit(X_train,y_train,epochs=50,validation_split=0.2,callbacks=[early_stopping])

In [ ]:
# Hyprameter
IMG_IND=224
IMG_SMAPE=(IMG_IND,IMG_IND,3)
LOSS=SparseCategoricalCrossentropy
IMG_SIZE=(224,224)
SEED=2
BATCH_SIZE=16


In [ ]:
def get_callback(model_name):
  callbacks=[]
  checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=f"model.{model_name}.h5",save_best_only=True)
  callbacks.append(checkpoint)
  anne=ReduceLROnplateau(monitor="val_loss",factor=0.5,patience=5,verbose=2,min_lr=0.0000001,min_delta=0.0001)
  callbacks.append(anne)
  earlystop=tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)
  callbacks.append(earlystop)
  return callbacks


In [ ]:
OPTIMIZER=tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999)

# DENSE MODEL
basel_model=tf.keras.applications.ResNet152(,include_top=False,weights="imagenet",input_shape=IMG_SMAPE)
basel_model.trainable=True
x=basel_model.output
x=GlobalAveragePooling2D()(x)
predictions=Dense(1,activation="sigmoid",name="Final")(x)
model=Model(inputs=basel_model.input,outputs=predictions)
model.compile(optimizer=OPTIMIZER,loss="categorical_crossentropy",metrics=["accuracy",keras.metrics.Percision() ,keras.metrics.Recall()])


In [ ]:
print(model.summary())

In [ ]:
callbacks=get_callback("resnet152")
history=model.fit(X_train,y_train,validation_split=0.2,batch_size=BATCH_SIZE,epochs=50,callbacks=[callbacks])

In [ ]:
result=model.evaluate(X_test,y_test)

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(['train','test'],loc='lower right')
plt.tight_layout()

NameError: ignored

In [ ]:
result=model.evaluate(X_test,y_test)
print("Test Loss",result[0])
print("Test Accuracy",result[1])